In [86]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

In [87]:
df = pd.read_csv("TRY4Full2013_2022Data.csv")
#pd.reset_option('display.max_columns')
pd.set_option('display.max_columns', None)

df.head()


,date,time,company name,district (2010),panel,ocs region,district name,area name,block,lease,water depth (feet),distance to shore (miles),structure name,structure type,rig name,rig type,pipeline segment number,incident summary,number fatalities,number injury > 3 days lost time,number injury > 3 days restricted work/job transfer,number injury 1-3 days lost time,number injury 1-3 days restricted work/job transfer,number of injury no lost time,explosion,fire,loss of well control (underground),loss of well control (surface),loss of well control (diverter),loss of well control (equipment failure/improper procedure),major collision (property damage > $25k),minor collision (property damage < S25k),crane,other lifting device,other lifting device name,reportable h2s release,shut down gas release,required muster,exploration,development production,drilling,workover,completion,motor vessel,pipeline,helicopter,other,other operation description,equipment failure,human error,slip trip fall,weather,external damage,leak,upset h2o,overboard fluid,other cause,other cause description,year
0,41183,1300.0,Shell Offshore Inc.,N,NaN,Gulf of Mexico,Houma,WALKER RIDGE 60,95,G31943,NaN,NaN,NaN,NaN,NOBLE GLOBETROTTER,Drillship,NaN,While using a hammer to remove a tong die from...,NaN,NaN,NaN,NaN,NaN,1.0,N,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,Y,Y,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,NaN,2013
1,41184,2000.0,Stone Energy Corporation,N,NaN,Gulf of Mexico,New Orleans,MAIN PASS AREA 17,288,G01665,420,35,A,Fixed Leg Platform,NaN,NaN,NaN,An individual was installing a man way cover b...,NaN,NaN,NaN,1.0,NaN,NaN,N,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,Y,N,N,N,N,N,N,Y,Vessel Cleaning,N,N,N,N,N,N,N,N,N,NaN,2013
2,41184,1645.0,ATP Oil & Gas Corporation,N,NaN,Gulf of Mexico,New Orleans,MISSISSIPPI CANYON 60,711,G14016,3005,49,A (GOMEZ),Semi Submersible (Column Stabilized Unit) Floa...,NaN,NaN,NaN,An employee was cleaning the inside of the wel...,NaN,NaN,NaN,NaN,NaN,1.0,N,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,Y,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,NaN,2013
3,41185,430.0,"Venoco, Inc.",N,NaN,Pacific,California,Los Angeles,6912,P00205,739,11,GAIL,Fixed Leg Platform,NaN,NaN,NaN,The GDE-701 gas detector head spiked to 20% LE...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,NaN,N,N,Y,N,Y,N,N,N,N,N,N,N,NaN,Y,N,N,N,N,N,N,N,N,NaN,2013
4,41185,815.0,Shell Offshore Inc.,N,NaN,Gulf of Mexico,Houma,WALKER RIDGE 60,508,G17001,NaN,NaN,NaN,NaN,NOBLE DANNY ADKINS,DP Semisubmersible,NaN,The crew was in the process of changing out th...,NaN,NaN,NaN,NaN,NaN,1.0,N,N,N,N,N,N,N,N,N,Y,Chain Hoist,N,N,N,N,Y,Y,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,NaN,2013


In [88]:
df_subset = df.dropna(subset=["number fatalities",
                              "number injury > 3 days lost time",
                              "number injury > 3 days restricted work/job transfer",
                              "number injury 1-3 days lost time",
                              "number injury 1-3 days restricted work/job transfer",
                              "number of injury no lost time"])

df_subset["Fatalities_and_Injury"] = df_subset[["number fatalities",
                              "number injury > 3 days lost time",
                              "number injury > 3 days restricted work/job transfer",
                              "number injury 1-3 days lost time",
                              "number injury 1-3 days restricted work/job transfer",
                              "number of injury no lost time"]].sum(axis=1)

df_subset["Cat_Fatalities_and_Injury"] = np.where(df_subset["Fatalities_and_Injury"] >= 1, "Yes", "No")


# Columns to remove
columns_to_remove = [
    'Fatalities_and_Injury',
    'number of injury no lost time',
    'number injury > 3 days restricted work/job transfer',
    'number injury > 3 days lost time'
]

# Drop the columns
df_subset = df_subset.drop(columns=columns_to_remove)

df_subset.tail()
#df_subset.to_csv("Data_for_prediction", index=False)


/var/folders/fq/n8rn1dtx6k34rv37c81hq01w0000gn/T/ipykernel_86957/3440038833.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["Fatalities_and_Injury"] = df_subset[["number fatalities",
/var/folders/fq/n8rn1dtx6k34rv37c81hq01w0000gn/T/ipykernel_86957/3440038833.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["Cat_Fatalities_and_Injury"] = np.where(df_subset["Fatalities_and_Injury"] >= 1, "Yes", "No")


,date,time,company name,district (2010),panel,ocs region,district name,area name,block,lease,water depth (feet),distance to shore (miles),structure name,structure type,rig name,rig type,pipeline segment number,incident summary,number fatalities,number injury 1-3 days lost time,number injury 1-3 days restricted work/job transfer,explosion,fire,loss of well control (underground),loss of well control (surface),loss of well control (diverter),loss of well control (equipment failure/improper procedure),major collision (property damage > $25k),minor collision (property damage < S25k),crane,other lifting device,other lifting device name,reportable h2s release,shut down gas release,required muster,exploration,development production,drilling,workover,completion,motor vessel,pipeline,helicopter,other,other operation description,equipment failure,human error,slip trip fall,weather,external damage,leak,upset h2o,overboard fluid,other cause,other cause description,year,Cat_Fatalities_and_Injury
6026,44924,1400.0,Shell Offshore Inc.,N,NaN,Gulf of Mexico,Lake Jackson,Alaminos Canyon,857,G17565,7835,140,NaN,NaN,H&P 205,Platform,NaN,"Rig Details: GA 021 ST01BP00, Lease - G24593, ...",0.0,0.0,0.0,N,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,Y,N,N,Y,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,NaN,2022,Yes
6027,44924,915.0,Chevron U.S.A. Inc.,N,NaN,Gulf of Mexico,Houma,Green Canyon,205,G05911,2200,80,A-Genesis Spar,SPAR,NaN,NaN,NaN,The crane operator received directions from th...,0.0,0.0,0.0,N,N,N,N,N,N,N,N,Y,N,NaN,N,N,N,N,Y,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,NaN,2022,No
6028,44925,2130.0,BP Exploration & Production Inc.,N,NaN,Gulf of Mexico,Houma,Green Canyon,743,G15607,6863,122,NaN,NaN,DIAMOND WEST AURIGA,Drillship,NaN,Crew was in the process of running VACS tool a...,0.0,0.0,0.0,N,N,N,N,N,N,N,N,N,Y,Elevators,N,N,N,N,Y,N,N,Y,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,NaN,2022,No
6029,44926,1230.0,Hess Corporation,N,NaN,Gulf of Mexico,Houma,Green Canyon,468,G26313,3360,109,A (STAMPEDE),TLP,NaN,NaN,NaN,Initial Report:\r\nSubject: [EXTERNAL] Hess - ...,0.0,0.0,0.0,N,Y,N,N,N,N,N,N,N,N,NaN,N,N,N,N,Y,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,NaN,2022,No
6031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,12.0,7.0,1,126,1,3,0,1,4,2,283,50,NaN,3,54,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,Yes


In [89]:
df_subset.shape

(2945, 57)

In [90]:
# Random forest model 

# Preprocess the data
df_subset = df_subset.dropna(subset=['Cat_Fatalities_and_Injury'])


# Extract features and target variable from the training set
X = df_subset.drop(columns=['Cat_Fatalities_and_Injury'])
y = df_subset["Cat_Fatalities_and_Injury"]

# Identify categorical columns in the dataset
categorical_cols = X.select_dtypes(include=['object']).columns

# Identify quantative columns in the dataset
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Split the data into training and testing sets- 80% is train, and 20% is test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Create transformers for numerical and categorical columns

# numerical- replace NA's with mean and scale the data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# categorical- replace NA's with the most frequent value and ignore any new value in the 
# test data that has not appeared in the training set.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Create a preprocessor to apply transformers to appropriate columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


# Apply preprocessing steps to the data
preprocessed_data = preprocessor.fit_transform(X)



In [91]:
# using Cross-Validation to tune the hyperparameters of RF model

param_grid = {
    'classifier__n_estimators': [75, 100, 150],
    'classifier__max_depth': [2, 5, 10],
    'classifier__min_samples_split': [14, 16, 18],
    'classifier__min_samples_leaf': [2, 3, 6]
}


# Create a pipeline with the preprocessor and Random Forest model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', rf_model)
])


# Instantiate GridSearchCV with the pipeline and parameter grid
grid_search = GridSearchCV(model,  # Use the entire pipeline here
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='accuracy', 
                           n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score found
print("Best Parameters:", grid_search.best_params_)



ValueError: Invalid parameter 'max_depth' for estimator Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['time', 'pipeline segment number', 'number fatalities',
       'number injury > 3 days lost time',
       'number injury > 3 days restricted work/job transfer',
       'number injury 1-3 days lost time',
       'number injury 1-3 days restricted work/job tran...
       'completion', 'motor vessel', 'pipeline', 'helicopter', 'other',
       'other operation description', 'equipment failure', 'human error',
       'slip trip fall', 'weather', 'external damage', 'leak

In [ ]:
# Get the best estimator
best_model_RF = grid_search.best_estimator_

# Tuned RF model's accuracy on train and test
y_train_pred = best_model_RF.predict(X_train)
y_test_pred = best_model_RF.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")

# Additional performance metrics
print("\nTraining Classification Report:")
print(classification_report(y_train, y_train_pred))

print("Testing Classification Report:")
print(classification_report(y_test, y_test_pred))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred))

In [ ]:
# Calculate AUC
y_test_prob = best_model_RF.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_test_prob)
print(f"AUC: {auc:.2f}")




In [ ]:
import numpy as np

# Access the Random Forest model from the Pipeline
random_forest_model = best_model_RF.named_steps['classifier']  # Ensure 'regressor' is the correct step name

# Access the feature importances
feature_importances = random_forest_model.feature_importances_

# Get the preprocessor from the pipeline
preprocessor = best_model_RF.named_steps['preprocessor']

# Get the indices of the top 25 features
top_feature_indices = np.argsort(feature_importances)[::-1][:25]

# Get the one-hot encoded feature names for categorical columns
categorical_cols = preprocessor.transformers_[1][2]
one_hot_encoded_feature_names = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_cols)

# Get the numerical column names
numerical_cols = preprocessor.transformers_[0][2]

# Combine numerical and one-hot encoded feature names
all_feature_names = list(numerical_cols) + list(one_hot_encoded_feature_names)

# Get the names of the top features
top_feature_names = [all_feature_names[i] for i in top_feature_indices]

# Print the top feature names
print("Top 25 features:")
print(top_feature_names)


In [28]:
# Random forest model 

# Preprocess the data
df_rf = df.dropna(subset=['number fatalities'])


# Extract features and target variable from the training set
X = df_rf.drop(columns=['number fatalities'])
y = df_rf["number fatalities"]

# Identify categorical columns in the dataset
categorical_cols = X.select_dtypes(include=['object']).columns

# Identify quantative columns in the dataset
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Split the data into training and testing sets- 80% is train, and 20% is test data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Create transformers for numerical and categorical columns

# numerical- replace NA's with mean and scale the data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# categorical- replace NA's with the most frequent value and ignore any new value in the 
# test data that has not appeared in the training set.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Create a preprocessor to apply transformers to appropriate columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


# Apply preprocessing steps to the data
preprocessed_data = preprocessor.fit_transform(X)



KeyError: ['Cat_Fatalities_and_Injury']

In [38]:
# using Cross-Validation to tune the hyperparameters of RF model

param_grid = {
    'regressor__n_estimators': [75, 100, 150],
    'regressor__max_depth': [2, 5, 10],
    'regressor__min_samples_split': [14, 16, 18],
    'regressor__min_samples_leaf': [2,3,6]
}

# Create a pipeline with the preprocessor and Random Forest model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', rf_model)
])

# Instantiate GridSearchCV with the pipeline and parameter grid
grid_search = GridSearchCV(model,  # Use the entire pipeline here
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score found
print("Best Parameters:", grid_search.best_params_)

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

...

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/meetpatel/opt/anaconda3/envs/ML/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213,

Best Parameters: {'regressor__max_depth': 2, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 14, 'regressor__n_estimators': 75}


In [27]:
# Best model

best_model_RF = grid_search.best_estimator_

# Tuned RF model's accuracy on train and test
y_train_pred = best_model_RF.predict(X_train)
y_test_pred = best_model_RF.predict(X_test)

RMSE_train = np.sqrt(MSE(y_train, y_train_pred))
RMSE_test = np.sqrt(MSE(y_test, y_test_pred))

print(f"Training RMSE score: {RMSE_train}")
print(f"Testing RMSE score: {RMSE_test}")

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"\nTraining R^2 score: {np.round(r2_train*100,2)}%")
print(f"Testing R^2 score: {np.round(r2_test*100,2)}%")

Training RMSE score: 0.0859530315400827
Testing RMSE score: 0.08245000425615241

Training R^2 score: 49.68%
Testing R^2 score: 49.39%


In [28]:
# Select the top 25 features that are automatically selected by RF

# Get the Random Forest model from the Pipeline
random_forest_model = best_model_RF.named_steps['regressor']

# Access the feature importances
feature_importances = random_forest_model.feature_importances_

# Get the preprocessor from the pipeline
preprocessor = best_model_RF.named_steps['preprocessor']

# Get the indices of the top features
top_feature_indices = np.argsort(feature_importances)[::-1][:25]

# Get the categorical columns
categorical_cols = preprocessor.transformers_[1][2]

# Get the one-hot encoded feature names for categorical columns (creating dummy variables)
one_hot_encoded_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_cols)

# Get the numerical column names
numerical_cols = preprocessor.transformers_[0][2]

# Combine numerical and one-hot encoded feature names
all_feature_names = list(numerical_cols) + list(one_hot_encoded_feature_names)

# Get the names of the top features
top_feature_names = [all_feature_names[i] for i in top_feature_indices]

# Print the top feature names
print("Top 25 features:")
print(top_feature_names)

Top 25 features:
['panel_N', 'panel_Y', 'year', 'number injury > 3 days lost time', 'lease_G02937', 'water depth (feet)_184', 'incident summary_An occupational fatality occurred.', 'distance to shore (miles)_123', 'block_109', 'date_44270', 'date_44273', 'date_44235', 'date_44002', 'date_43844', 'date_43984', 'date_44417', 'time', 'date_44494', 'loss of well control (surface)_N', 'lease_G01313', 'block_215', 'block_142', 'loss of well control (underground)_N', 'water depth (feet)_6267', 'distance to shore (miles)_89']


In [ ]:
# Variables for further analysis: 

# water depth, distance to shore, incident summary, 
# loss of well control (surface), lease, loss of well control (underground)

In [34]:
# XG Boost

from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Assume df contains your dataframe


# Preprocess the data
df_rf = df.dropna(subset=['number fatalities'])

# Extract features and target variable from the training set
X = df_rf.drop(columns=['number fatalities'])
y = df_rf["number fatalities"]

# Identify categorical columns in the dataset
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(exclude=['object']).columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = XGBRegressor()

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)


rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print("RMSE on training set:", rmse_train)
print("RMSE on test set:", rmse_test)

# Calculate R^2 score for training set
r2_train = r2_score(y_train, y_pred_train)

# Calculate R^2 score for test set
r2_test = r2_score(y_test, y_pred_test)

print("R^2 score on training set:", np.round(r2_train*100,1),"%")
print("R^2 score on test set:", np.round(r2_test*100,1),"%")

ModuleNotFoundError: No module named 'xgboost'